In [1]:
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random

# Data Preprocessing for Matrix User x Item

In [2]:
#Source Code for data Processing
#https://gist.github.com/victorkohler/f48ea6512058719ba52053851fedc745

train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

In [3]:
#np.shape(train_data)
np.shape(test_data)

(3782335, 12)

In [4]:
#train = train_data.append(test_data)
#np.shape(train)

In [5]:
item_metadata = pd.read_csv('data/item_metadata.csv', sep=',', engine='python')

In [6]:
train_v2 = train_data[train_data['action_type']=='clickout item']
train_v2= train_v2[train_v2.reference.notnull()]
train_v2.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
29,01R92KASN25O,62991f7c78f27,1541055672,8,clickout item,7818446,DE,"Luebeck, Germany",mobile,NaN,7818446|51315|2133708|119638|86359|133581|6688...,163|75|93|98|169|201|129|117|108|170|133|80|87...
32,026J2T90PT57,67c4d45f56146,1541105688,1,clickout item,2681512,MX,"Ezequiel Montes, Mexico",mobile,NaN,6010044|5433188|5156000|4780400|5137162|992819...,38|35|54|37|73|37|37|30|71|75|64|26|25|37|40|3...
39,032W52V15J3A,e84393cf62d13,1541102603,7,clickout item,929533,HR,"Cancun, Mexico",mobile,NaN,3132957|100226|1954167|9462680|2776177|929533|...,25|324|67|52|57|21|90|240|64|37|70|60|167|45|6...
73,03F93632Y45U,f09aa370de0af,1541109160,34,clickout item,8496296,US,"San Diego, USA",mobile,NaN,9112592|102414|9140448|5654906|4341120|9739716...,263|114|263|273|122|194|236|161|219|113|110|19...
393,08527D0MD8VC,3e2a95e5727f9,1541099104,320,clickout item,8266350,ES,"Lisbon, Portugal",desktop,NaN,3505518|1834297|4837106|3990396|2042495|486747...,47|50|45|40|45|40|40|50|50|45|50|45|38|49|48|3...


In [7]:
train_v2.dtypes

user_id            object
session_id         object
timestamp           int64
step                int64
action_type        object
reference          object
platform           object
city               object
device             object
current_filters    object
impressions        object
prices             object
dtype: object

In [8]:
train_v2["reference"]= train_v2["reference"].astype(int)
train_v2["step"]= train_v2["step"].astype(int)
train_v2.dtypes

user_id            object
session_id         object
timestamp           int64
step                int64
action_type        object
reference           int64
platform           object
city               object
device             object
current_filters    object
impressions        object
prices             object
dtype: object

In [ ]:
train_v2.head()
#np.shape(train_v2)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
29,01R92KASN25O,62991f7c78f27,1541055672,8,clickout item,7818446,DE,"Luebeck, Germany",mobile,NaN,7818446|51315|2133708|119638|86359|133581|6688...,163|75|93|98|169|201|129|117|108|170|133|80|87...
32,026J2T90PT57,67c4d45f56146,1541105688,1,clickout item,2681512,MX,"Ezequiel Montes, Mexico",mobile,NaN,6010044|5433188|5156000|4780400|5137162|992819...,38|35|54|37|73|37|37|30|71|75|64|26|25|37|40|3...
39,032W52V15J3A,e84393cf62d13,1541102603,7,clickout item,929533,HR,"Cancun, Mexico",mobile,NaN,3132957|100226|1954167|9462680|2776177|929533|...,25|324|67|52|57|21|90|240|64|37|70|60|167|45|6...
73,03F93632Y45U,f09aa370de0af,1541109160,34,clickout item,8496296,US,"San Diego, USA",mobile,NaN,9112592|102414|9140448|5654906|4341120|9739716...,263|114|263|273|122|194|236|161|219|113|110|19...
393,08527D0MD8VC,3e2a95e5727f9,1541099104,320,clickout item,8266350,ES,"Lisbon, Portugal",desktop,NaN,3505518|1834297|4837106|3990396|2042495|486747...,47|50|45|40|45|40|40|50|50|45|50|45|38|49|48|3...


In [ ]:
#train_v3 = train_v2.head(10000)
train_v3 = train_v2
train_v3.head()
np.shape(train_v3)

(1586586, 12)

In [ ]:
data = train_v3[['user_id','reference']]
data.head()
#np.shape(data)

,user_id,reference
29,01R92KASN25O,7818446
32,026J2T90PT57,2681512
39,032W52V15J3A,929533
73,03F93632Y45U,8496296
393,08527D0MD8VC,8266350


In [ ]:
data=data.groupby(['user_id','reference']).size().reset_index()
data.columns = ['user', 'item', 'click_count']
data.head()

,user,item,click_count
0,0001VQMGUI65,477811,1
1,0001VQMGUI65,950829,1
2,0001VQMGUI65,2019467,1
3,0001VQMGUI65,3133074,2
4,0003QTCX5MJX,2195060,1


In [ ]:
user_id_table = data['user']
user_id_table = pd.DataFrame(user_id_table)
user_id_table = pd.DataFrame(user_id_table['user'].unique())
user_id_table.columns = ['user']
user_id_table['user_id'] = dict(enumerate(user_id_table['user'], start=0))
user_id_table.head()
#np.shape(user_id_table)

,user,user_id
0,0001VQMGUI65,0
1,0003QTCX5MJX,1
2,0004IOZI7CKF,2
3,0004WCFRV3FB,3
4,0006W0R5A5V8,4


In [ ]:
item_id_table = data['item']
item_id_table = pd.DataFrame(item_id_table)
item_id_table = pd.DataFrame(item_id_table['item'].unique())
item_id_table.columns = ['item']
item_id_table['item_id'] = dict(enumerate(item_id_table['item'], start=0))
item_id_table.head()
#np.shape(item_id_table)

,item,item_id
0,477811,0
1,950829,1
2,2019467,2
3,3133074,3
4,2195060,4


In [ ]:
data = data.merge(user_id_table, how='left', left_on='user', right_on = 'user').merge(item_id_table, how='left', left_on='item', right_on = 'item')
data.head()

,user,item,click_count,user_id,item_id
0,0001VQMGUI65,477811,1,0,0
1,0001VQMGUI65,950829,1,0,1
2,0001VQMGUI65,2019467,1,0,2
3,0001VQMGUI65,3133074,2,0,3
4,0003QTCX5MJX,2195060,1,1,4


# Graph Embedding

In [ ]:
#Author Manuel Aguero
import scipy.sparse as sparse
import numpy as np
import random
import mpmath
import time

def mini_batch_generator(graph, batch_size, row_length, col_length):
    i = random.randint(row_length-batch_size)
    j = random.randint(col_length-batch_size)
    return graph[i:i+batch_size,j:j+batch_size]

def negative_sampling(E, row_length, col_length, batch_size):
    S = sparse.dok_matrix((row_length + 1, col_length + 1)) #create empty negtive feedack graph
    for i in range(batch_size):
        a = random.randint(row_length)
        b = random.randint(col_length)
        randomEdge = (a,b) #randomly sample edges from the implicit feedback graph
        if (a,b) not in E:
            S[a,b]=1
    return S.tocoo()

def compute_e_ife(theta_u, theta_v, G_items, S_items):
    sumG = sumS = 0
    for (i,j) in G_items:
        sumG += np.log(1/(1+np.exp(np.dot(-theta_u[i], np.transpose(theta_v[j])))))
    for (i,j) in S_items:
        sumS += np.log(1 - (1/(1+np.exp(np.dot(-theta_u[i], np.transpose(theta_v[j]))))) )
    return sumG + sumS
            
def update_embeddings(theta_u, theta_v, G_mini_items, S_items, a, e_ife):
    sumGu = sumSu = sumGv = sumSv = 0
    for (i,j) in G_mini_items:
        currVal = (1 - 1/(1+np.exp(-np.dot(theta_u[i], np.transpose(theta_v[j])))))
        uVal = np.dot(currVal, theta_v[j])  
        vVal = np.dot(currVal, theta_u[i])
        sumGu += uVal
        sumGv += vVal
    for (i,j) in S_items:
        currVal = (-1/(1+np.exp(-np.dot(theta_u[i],np.transpose(theta_v[j])))))
        uVal = np.dot(currVal, theta_v[j])  
        vVal = np.dot(currVal, theta_u[i])
        sumSu += uVal
        sumSv += vVal
    theta_u = theta_u + (a*sumGu + a*sumSu) #update user embeddings
    theta_v = theta_v + (a*sumGv + a*sumSv) #update item embeddings
    new_eife = compute_e_ife(theta_u, theta_v, G_mini_items, S_items) #check S: expand S or use new mini S?
    difference = new_eife - e_ife
    if e_ife > new_eife:
        theta_u = theta_u - (a*sumGu + a*sumSu) #undo
        theta_v = theta_v - (a*sumGv + a*sumSv) #undo
    else:
        e_ife = new_eife
    return (difference, e_ife)
    
def implicit_feedback_embedding(G, b, a, k, row_length, col_length, tolerance):
    #initiliase list of vector embeddings randomly
    theta_u = list()
    theta_v = list()
    for i in range(row_length + 1):
        theta_u.append(np.random.rand(k))
    for i in range(col_length + 1):
        theta_v.append(np.random.rand(k))
    convergence_difference = 100 #arbitrary to initialisation
    G_items = set(zip(G.row, G.col)) #get iterator for row,col pair with non-zero values
    e_ife = None
    G = G.tocsr()
    print(theta_u[0], theta_v[0])
    start = time.time()
    while abs(convergence_difference) > tolerance: #while embedding has not converged
        # If algorithm has used more than five minutes and not converged yet, move on to the next parameters
        if time.time() > start + 300:
            print("Did not converge in five minutes")
            break
        print("diff > tolerance?",abs(convergence_difference),tolerance)
        G_mini = mini_batch_generator(G, b, row_length, col_length)
        S = negative_sampling(G_items, row_length, col_length, b) #make negative feedback graph
        G_mini = G.tocoo()
        S_items = set(zip(S.row, S.col))
        G_mini_items = set(zip(G_mini.row, G_mini.col))
        if not e_ife:
            e_ife = compute_e_ife(theta_u, theta_v, G_mini_items, S_items) #initialise e_ife mini G or G?
        convergence_difference, e_ife = update_embeddings(theta_u, theta_v, G_mini_items, S_items, a, e_ife)
    print("diff > tolerance?", convergence_difference, tolerance)
    print(theta_u[0], theta_v[0])
 

learning_rates = [0.01, 0.001, 0.0001]
dims = [1, 2, 5, 10, 20, 40]
tolerances = [0.1, 1, 5, 10, 20]
batch_sizes = [500, 2000, 10000, 20000]

for lr in learning_rates:
    for dim in dims:
        for tol in tolerances:
            for bs in batch_sizes:
                print()
                print("Running with parameters:")
                print("Learning rate: " + str(lr))
                print("Dimentions: " + str(dim))
                print("Tolerance: " + str(tol))
                print("Batch size: " + str(bs))
                print()
                G = sparse.csr_matrix((data['click_count'].astype(float), (data['user_id'], data['item_id'])))#make user/item graph
                G = G[:100000,:100000]
                G = G.tocoo()
                row_length = max(G.row)
                col_length = max(G.col)
                print("embedding...")
                implicit_feedback_embedding(G,bs,lr,dim,row_length,col_length, tol)
                print("done")


Running with parameters:
Learning rate: 0.01
Dimentions: 1
Tolerance:0.1
Batch size: 500

embedding...
[0.33119875] [0.67115468]
diff > tolerance? 100 0.1


/home/maria/.local/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: divide by zero encountered in log


diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > toler

diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > toler

diff > tolerance? inf 1
diff > tolerance? inf 1
diff > tolerance? inf 1
diff > tolerance? inf 1
diff > tolerance? inf 1
diff > tolerance? inf 1
diff > tolerance? inf 1
diff > tolerance? inf 1
diff > tolerance? inf 1
diff > tolerance? inf 1
diff > tolerance? inf 1
diff > tolerance? inf 1
diff > tolerance? inf 1
diff > tolerance? inf 1
diff > tolerance? inf 1
diff > tolerance? inf 1
diff > tolerance? inf 1
diff > tolerance? inf 1
diff > tolerance? inf 1
diff > tolerance? inf 1
diff > tolerance? inf 1
diff > tolerance? inf 1
diff > tolerance? inf 1
diff > tolerance? inf 1
diff > tolerance? inf 1
diff > tolerance? inf 1
diff > tolerance? inf 1
Did not converge in five minutes
diff > tolerance? -inf 1
[0.5692254] [0.37638552]
done

Running with parameters:
Learning rate: 0.01
Dimentions: 1
Tolerance:1
Batch size: 20000

embedding...
[0.01334766] [0.16176809]
diff > tolerance? 100 1
diff > tolerance? inf 1
diff > tolerance? inf 1
diff > tolerance? inf 1
diff > tolerance? inf 1
diff > toleran

...

diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance? inf 5
diff > tolerance

diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10
diff > tolerance? inf 10


diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20


diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
diff > tolerance? inf 20
Did not converge in five minutes
diff > tolerance? -inf 20
[0.4443717 0.5268472] [0.79961206 0.99640041]
done

Running with parameters:
Learning rate: 0.01
Dimentions: 2
Tolerance:20
Batch size: 20000

embedding...
[0.31845103 0.91912963] [0.83661179 0.56316934]
diff > toler

diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
Did not converge in five minutes
diff > tolerance? -inf 0.1
[0.47977293 0.34082088 0.68531358 0.36977675 0.47336057] [0.51451651 0.83915788 0.16360524 0.01230657 0.33336016]
done

Running with parameters:
Learning rate: 0.01
Dimentions: 5
Tolerance:0.1
Batch size: 10000

embedding...
[0.40003784 0.57113818 0.23650176 0.79785508 0.18157911] [0.07484017 0.34747633 0.86159045 0.31576742 0.20932793]
diff > tolerance? 100 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
diff > tolerance? inf 0.1
dif